In [1]:
# TDAcancer_detect function

In [2]:
# Cancer Detection

In [3]:
import pandas as pd
import numpy as np
import os
import gudhi
from sklearn.linear_model import LinearRegression

def TDAcancer_detect_1(file_path, biomarkers, random_patient_range, control_data_range, regression_threshold, distance_threshold, max_life_range_threshold_upper, max_life_range_threshold_lower, tumor_types):
    # Step 1: Load the Excel data
    df_1 = pd.read_excel(file_path, sheet_name=sheet_name)
    df_2 = pd.read_excel(file_path, sheet_name=sheet_name)

    # Step 2: Select the biomarkers (user-defined)
    df_1 = df_1[biomarkers].iloc[:59]  # Fixed dataset of 59 normal patients
    df_2 = df_2[biomarkers]

    # Step 3: Initialize list for topological features
    data_list = []

    # Step 4: Loop through rows 60 to random_patient_range and perform TDA analysis
    for index in range(60, random_patient_range + 1):
        row_to_add = df_2.iloc[[index]]
        df_new = pd.concat([df_1, row_to_add], ignore_index=True)

        # Log normalize numeric columns
        numeric_cols = df_new.select_dtypes(include=[np.number]).columns
        df_new[numeric_cols] = np.log(df_new[numeric_cols])

        # Pearson correlation matrix
        corr_matrix = np.corrcoef(df_new.values, rowvar=False)
        distance_matrix = 1 - np.abs(pd.DataFrame(corr_matrix).values)

        # Compute persistence homology
        rips_complex = gudhi.RipsComplex(distance_matrix=distance_matrix)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
        persistence = simplex_tree.persistence()

        # Extract 1D and 0D hole statistics
        finite_persistence = [(dim, (birth, death)) for dim, (birth, death) in persistence if death != float('inf')]
        one_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 1]
        zero_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 0]

        max_life_range_1D = max((death - birth for birth, death in one_dim_holes), default=0)
        avg_birth_1D = np.mean([birth for birth, _ in one_dim_holes]) if one_dim_holes else 0
        avg_death_1D = np.mean([death for _, death in one_dim_holes]) if one_dim_holes else 0
        num_one_dim_holes = len(one_dim_holes)

        min_life_range_0D = min((death - birth for birth, death in zero_dim_holes), default=0)
        max_life_range_0D = max((death - birth for birth, death in zero_dim_holes), default=0)
        avg_death_0D = np.mean([death for _, death in zero_dim_holes]) if zero_dim_holes else 0
        num_zero_dim_holes = len(zero_dim_holes) + 1
        range_death_0D = max([death for _, death in zero_dim_holes], default=0) - min([death for _, death in zero_dim_holes], default=0)
        median_death_0D = np.median([death for _, death in zero_dim_holes]) if zero_dim_holes else 0
        std_death_0D = np.std([death for _, death in zero_dim_holes]) if zero_dim_holes else 0

        # Store feature vector
        data = {
            "0-Dim Hole Min Life Range": [min_life_range_0D],
            "0-Dim Hole Max Life Range": [max_life_range_0D],
            "Range_Death_0D": [range_death_0D],
            "Num_0D_Holes": [num_zero_dim_holes],
            "Median_Death_0D": [median_death_0D],
            "Std_Death_0D": [std_death_0D],
            "Avg_Death_0D": [avg_death_0D],
            "Average Birth (1D)": [avg_birth_1D],
            "Average Death (1D)": [avg_death_1D],
            "1-Dim Hole Max Life Range": [max_life_range_1D],
            "Number of 1-Dimensional Holes": [num_one_dim_holes]
        }
        df = pd.DataFrame(data, index=[f"P_{index}"])
        data_list.append(df)

    # Step 5: Save results to CSV
    final_df = pd.concat(data_list)

    # Add Patient ID, Sample ID, Cohort, and AJCC Stage to the final CSV
    additional_columns = ['Patient ID #', 'Cohort', 'AJCC Stage']
    df_clinical = pd.read_excel(file_path, sheet_name=sheet_name)
    df_additional_info = df_clinical[additional_columns].iloc[60:random_patient_range + 1]  # Corresponding rows to additional patients
    additional_info_index = [f"P_{i}" for i in range(60, random_patient_range + 1)]
    df_additional_info.index = additional_info_index

    # Merge the additional columns with the TDA results
    df_combined = pd.concat([df_additional_info, final_df], axis=1)

    # Get directory from original file path and create consistent output filename
    directory = os.path.dirname(file_path)
    csv_file_path = os.path.join(directory, "Clinical cancer data_features.csv")
    df_combined.to_csv(csv_file_path, index=True)

    # Step 6: Train Linear Regression Model
    new_data = pd.read_csv(csv_file_path)
    control_data = new_data.iloc[:control_data_range]  # User-defined control data range

    X = control_data[['0-Dim Hole Min Life Range', '0-Dim Hole Max Life Range', 'Median_Death_0D', 'Std_Death_0D']]
    y = control_data['Avg_Death_0D']
    model = LinearRegression().fit(X, y)

    # Step 7: Predict and Analyze
    def predict_Avg_Death_0D(row):
        return np.dot(model.coef_, [row[col] for col in X.columns]) + model.intercept_

    new_data['Predicted Avg_Death_0D'] = new_data.apply(predict_Avg_Death_0D, axis=1)
    new_data['Difference'] = new_data['Avg_Death_0D'] - new_data['Predicted Avg_Death_0D']
    new_data['Regression Difference Percentage'] = abs(new_data['Difference'] / new_data['Avg_Death_0D']) * 100

    reference_point = (control_data['Average Birth (1D)'].mean(), control_data['Average Death (1D)'].mean())

    new_data['Distance %(1)'] = new_data.apply(
        lambda row: np.sqrt((row['Average Birth (1D)'] - reference_point[0]) ** 2 + 
                    (row['Average Death (1D)'] - reference_point[1]) ** 2) * 100,
        axis=1
    )

    new_data['Result(Cancer)'] = new_data.apply(
        lambda row: 'Positive' if sum([
            row['Regression Difference Percentage'] > k_1,
            row['Distance %(1)'] > k_2,
            row['1-Dim Hole Max Life Range'] > k_3,
            row['1-Dim Hole Max Life Range'] < k_4
        ]) >= 2 else 'Negative',
        axis=1
    )

    new_data.to_csv(csv_file_path, index=False)

    # Step 8: Count positive results per cohort
    positive_counts = new_data[new_data['Result(Cancer)'] == 'Positive'].groupby('Cohort').size()

    # Filter the results based on user-defined cancer types
    results = {}
    for tumor_type in tumor_types:
        results[f"{tumor_type} Positive"] = positive_counts.get(tumor_type, 0)

    # Calculate total positive cancers excluding 'Normal'
    results["Total Positive Cancers"] = sum(positive_counts.values) - positive_counts.get('Normal', 0)

    return results

# Example usage:
file_path = r'D:\Cancer Detection folder\Test\Clinical cancer data.xlsx'
sheet_name = 'Normal and Cancer'
# User-defined variables
biomarkers = ['Angiopoietin-2', 'CA-125', 'CA 15-3', 'CEA', 'CYFRA 21-1', 'FGF2', 'G-CSF', 'HE4', 'HGF', 'PAR', 'sPECAM-1', 'Thrombospondin-2']
random_patient_range = 1802  # User-defined range
control_data_range = 554  # User-defined control data range
k_1 = 0.444  # User-defined regression threshold
k_2 = 2.3  # User-defined distance threshold
k_3 = 0.112  # User-defined upper threshold for 1D hole max life range
k_4 = 0.047  # User-defined lower threshold for 1D hole max life range
tumor_types = ['Normal', 'Breast', 'Colorectum', 'Esophagus', 'Liver', 'Lung', 'Ovary', 'Pancreas', 'Stomach']  # User-defined tumor types

# Run the pipeline
results = TDAcancer_detect_1(
    file_path, biomarkers, random_patient_range, control_data_range, 
    k_1, k_2, k_3, k_4, tumor_types
)
print(results)

{'Normal Positive': 36, 'Breast Positive': 77, 'Colorectum Positive': 182, 'Esophagus Positive': 25, 'Liver Positive': 29, 'Lung Positive': 47, 'Ovary Positive': 41, 'Pancreas Positive': 42, 'Stomach Positive': 55, 'Total Positive Cancers': 498}


In [ ]:
# Cancer Tissue Localization

In [4]:
import os
import re
import pandas as pd
import numpy as np
import gudhi
from itertools import combinations

def TDAcancer_detect_2(file_path, sheet_name, biomarkers, output_dir, tumor_types, total_positive_patients, num_combinations, random_patient_range, cohorts_to_analyze):
    
    # ==================================================
    # Part 1: Preliminary Analysis and Formation of Average1.xlsx
    # ==================================================

    def calculate_averages(output_dir, tumor_types, total_positive_patients, num_combinations):
        # File paths
        cancer_result_file = os.path.join(output_dir, 'Clinical cancer data_features.csv')
        base_feature_vector_path = os.path.join(output_dir, 'FeatureVectorTissueDet{}.csv')
        base_average_path = os.path.join(output_dir, 'AverageDet{}.csv')

        # Load the cancer results file
        cancer_data = pd.read_csv(cancer_result_file)

        # Columns for averaging
        columns_to_average = ['0-Dim Hole Min Life Range', 'Range_Death_0D']

        # Process files from FeatureVectorTissueDet1.csv to FeatureVectorTissueDetN.csv
        for i in range(1, num_combinations + 1):
            file_path = base_feature_vector_path.format(i)
            try:
                new_data = pd.read_csv(file_path)
            except FileNotFoundError:
                print(f"File {file_path} not found, skipping.")
                continue
            
            # Add 'Result(Cancer)' column
            new_data['Result(Cancer)'] = cancer_data['Result(Cancer)']
            new_data.to_csv(file_path, index=False)
            
            # Compute cohort averages
            averages_list = []
            for tumor, row_limit in tumor_types.items():
                if tumor == 'normal':
                    filtered_data = new_data[(new_data['Cohort'].str.strip().str.lower() == tumor) & (new_data['Result(Cancer)'] == 'Negative')]
                else:
                    filtered_data = new_data[(new_data['Cohort'].str.strip().str.lower() == tumor) & (new_data['Result(Cancer)'] == 'Positive')]
                
                selected_data = filtered_data.iloc[:row_limit][columns_to_average].dropna()
                averages_list.append({'Cohort': tumor.capitalize(), **selected_data.mean().to_dict()})
            
            averages_data = pd.DataFrame(averages_list)
            normal_averages = averages_data[averages_data['Cohort'].str.lower() == 'normal']
            reference_point = (
                float(normal_averages['0-Dim Hole Min Life Range'].values[0]),
                float(normal_averages['Range_Death_0D'].values[0])
            )
            
            # Compute Euclidean distance
            X = new_data[['0-Dim Hole Min Life Range']].values
            y = new_data['Range_Death_0D'].values
            distances = [np.sqrt((X[i][0] - reference_point[0]) ** 2 + (y[i] - reference_point[1]) ** 2) for i in range(len(X))]
            new_data['Distance %(3)'] = [d * 100 for d in distances]
            new_data.to_csv(file_path, index=False)
            
            # Compute distance averages
            averages_distance = []
            for tumor, row_limit in tumor_types.items():
                if tumor == 'normal':
                    filtered_data = new_data[(new_data['Cohort'].str.strip().str.lower() == tumor) & (new_data['Result(Cancer)'] == 'Negative')]
                else:
                    filtered_data = new_data[(new_data['Cohort'].str.strip().str.lower() == tumor) & (new_data['Result(Cancer)'] == 'Positive')]
                
                selected_data = filtered_data.iloc[:row_limit][columns_to_average + ['Distance %(3)']].dropna()
                averages_distance.append({'Cohort': tumor.capitalize(), 'Average Distance %(3)': selected_data['Distance %(3)'].mean(), **selected_data.mean().to_dict()})
            
            averages_distance_data = pd.DataFrame(averages_distance)
            averages_distance_data.to_csv(base_average_path.format(i), index=False)
            
            # Determine threshold
            sorted_distances = averages_distance_data['Average Distance %(3)'].sort_values(ascending=False)
            second_highest_distance = sorted_distances.iloc[1]
            k = int(np.ceil(second_highest_distance))
            
            # Localize cancer tissue
            new_data['Result(Tissue Detection)'] = ['Positive' if value > k else 'Negative' for value in new_data['Distance %(3)']]
            new_data['Cancer tissue'] = new_data.apply(lambda row: 'Positive' if row['Result(Cancer)'] == 'Positive' and row['Result(Tissue Detection)'] == 'Positive' else 'Negative', axis=1)
            new_data.to_csv(file_path, index=False)
            
            print(f"Processed and updated {file_path} and {base_average_path.format(i)} successfully.")

    def merge_averages_to_excel(output_dir, num_combinations):
        # Function for natural sorting (e.g., 1,2,3 instead of 1,10,100)
        def natural_sort_key(text):
            return [int(num) if num.isdigit() else num for num in re.split(r'(\d+)', text)]

        # Merge AverageDet CSV files into Average1.xlsx
        # Get a sorted list of all relevant CSV files
        csv_files = sorted(
            [f for f in os.listdir(output_dir) if f.startswith("AverageDet") and f.endswith(".csv")],
            key=natural_sort_key
        )

        # Initialize an empty DataFrame for merging
        merged_df = None

        # Loop through each file and process
        for i, file in enumerate(csv_files, start=1):
            if i > num_combinations:
                break  # Stop after processing the specified number of combinations
            
            # Read the current CSV file
            file_path = os.path.join(output_dir, file)
            df = pd.read_csv(file_path)

            # Select only the "Cohort" and "Average Distance %(3)" columns
            df = df[["Cohort", "Average Distance %(3)"]]

            # Rename the "Average Distance %(3)" column to include the file index
            df.rename(columns={"Average Distance %(3)": f"Average Distance %(3)-{i}"}, inplace=True)

            # Merge with accumulated DataFrame
            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on="Cohort", how="outer")

        # Save the merged DataFrame to an Excel file
        output_file_path = os.path.join(output_dir, "Average1.xlsx")
        merged_df.to_excel(output_file_path, index=False, engine='openpyxl')
        print(f"Merged file saved as: {output_file_path}")

    def process_clinical_data(file_path, sheet_name, biomarkers, output_dir, tumor_types, total_positive_patients, num_combinations, random_patient_range):
        # Step 1: Load the clinical data
        df_1 = pd.read_excel(file_path, sheet_name=sheet_name)
        df_2 = pd.read_excel(file_path, sheet_name=sheet_name)

        # Generate all combinations of 3 biomarkers
        biomarker_combinations = list(combinations(biomarkers, 3))
        if num_combinations > len(biomarker_combinations):
            print(f"Warning: Only {len(biomarker_combinations)} combinations available. Processing {len(biomarker_combinations)} combinations instead of {num_combinations}.")
            num_combinations = len(biomarker_combinations)

        # Directory to save the CSV files
        os.makedirs(output_dir, exist_ok=True)

        # Step 2: Iterate through each combination of biomarkers
        for combo_index, combo in enumerate(biomarker_combinations[:num_combinations], start=1):
            # Select biomarkers for this iteration
            columns_to_select = list(combo)
            print(f"Processing combination {combo_index}: {columns_to_select}")

            # Subset the data
            df_1_subset = df_1[columns_to_select].iloc[:59]  # First 59 rows for normal patients
            df_2_subset = df_2[columns_to_select]           # All rows for cancer patients

            # Initialize a list to store feature vectors
            data_list = []

            # Step 4: Add additional random patient data for analysis
            for index in range(60, random_patient_range + 1):  # User-defined range
                # Select a specific row from df_2
                row_to_add = df_2_subset.iloc[[index]]

                # Combine it with the fixed dataset
                df_combined = pd.concat([df_1_subset] + [row_to_add] * 1, ignore_index=True)

                # Step 5: Log normalize the combined data
                numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
                df_combined[numeric_cols] = np.log(df_combined[numeric_cols])

                # Convert DataFrame to numpy array
                df_subset = df_combined.values

                # Step 6: Calculate Pearson correlation matrix and distance matrix
                corr_matrix = np.corrcoef(df_subset, rowvar=False)
                corr_matrix_df = pd.DataFrame(corr_matrix, columns=columns_to_select, index=columns_to_select)
                distance_matrix = 1 - np.abs(corr_matrix_df)
                distance_matrix_np = distance_matrix.values

                # Step 7: Construct a simplicial complex using Gudhi's RipsComplex
                rips_complex = gudhi.RipsComplex(distance_matrix=distance_matrix_np)
                simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
                persistence = simplex_tree.persistence()

                # Step 8: Extract topological features
                finite_persistence = [(dim, (birth, death)) for dim, (birth, death) in persistence if death != float('inf')]
                one_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 1]
                
                if one_dim_holes:
                    max_life_range_1D = max(death - birth for birth, death in one_dim_holes)
                    avg_birth_1D = np.mean([birth for birth, _ in one_dim_holes])
                    avg_death_1D = np.mean([death for _, death in one_dim_holes])
                    num_one_dim_holes = len(one_dim_holes)
                else:
                    max_life_range_1D = avg_birth_1D = avg_death_1D = num_one_dim_holes = 0

                zero_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 0]
                min_life_range_0D = min(death - birth for birth, death in zero_dim_holes)
                max_life_range_0D = max(death - birth for birth, death in zero_dim_holes)
                avg_death_0D = np.mean([death for _, death in zero_dim_holes])
                num_zero_dim_holes = len(zero_dim_holes) + 1
                range_death_0D = max([death for _, death in zero_dim_holes]) - min([death for _, death in zero_dim_holes])
                median_death_0D = np.median([death for _, death in zero_dim_holes])
                std_death_0D = np.std([death for _, death in zero_dim_holes])

                # Prepare feature vector
                data = {
                    "0-Dim Hole Min Life Range": [min_life_range_0D],
                    "0-Dim Hole Max Life Range": [max_life_range_0D],
                    "Range_Death_0D": [range_death_0D],
                    "Num_0D_Holes": [num_zero_dim_holes],
                    "Median_Death_0D": [median_death_0D],
                    "Std_Death_0D": [std_death_0D],
                    "Avg_Death_0D": [avg_death_0D],
                    "Average Birth (1D)": [avg_birth_1D],
                    "Average Death (1D)": [avg_death_1D],
                    "1-Dim Hole Max Life Range": [max_life_range_1D],
                    "Number of 1-Dimensional Holes": [num_one_dim_holes]
                }

                # Append to list
                df_features = pd.DataFrame(data, index=[f"P_{index}"])
                data_list.append(df_features)

            # Concatenate all feature vectors for the current combination
            final_df = pd.concat(data_list)

            # Add additional clinical information
            additional_columns = ['Patient ID #', 'Cohort', 'AJCC Stage']
            df_clinical = pd.read_excel(file_path, sheet_name=sheet_name)
            df_additional_info = df_clinical[additional_columns].iloc[60:random_patient_range + 1]
            additional_info_index = [f"P_{i}" for i in range(60, random_patient_range + 1)]
            df_additional_info.index = additional_info_index

            # Merge with topological data
            df_combined = pd.concat([df_additional_info, final_df], axis=1)

            # Append biomarkers as the last row
            selected_biomarkers_row = pd.DataFrame(
                {col: None for col in df_combined.columns}, 
                index=["Selected Biomarkers"]
            )
            selected_biomarkers_row.iloc[0, :len(columns_to_select)] = columns_to_select
            df_combined = pd.concat([df_combined, selected_biomarkers_row])

            # Save to a CSV file
            csv_file_path = os.path.join(output_dir, f"FeatureVectorTissueDet{combo_index}.csv")
            df_combined.to_csv(csv_file_path, float_format="%f", index=True)
            print(f"Saved: {csv_file_path}")

        # Step 9: Calculate averages and generate Average1.xlsx
        calculate_averages(output_dir, tumor_types, total_positive_patients, num_combinations)
        merge_averages_to_excel(output_dir, num_combinations)

    # ==================================================
    # Part 2: Formation of Positives1.xlsx and Sensitivity Outcomes
    # ==================================================

    def merge_positives_to_excel(output_dir, tumor_types, num_combinations):
        # Directory paths
        input_dir = output_dir
        positives_dir = os.path.join(input_dir, 'Positives')

        # Ensure the output directory exists
        os.makedirs(positives_dir, exist_ok=True)

        # Loop through files 1 to N
        for i in range(1, num_combinations + 1):
            input_file_path = os.path.join(input_dir, f'FeatureVectorTissueDet{i}.csv')
            output_file_path = os.path.join(positives_dir, f'positives_countDet{i}.csv')

            try:
                # Step 1: Load the CSV file
                new_data = pd.read_csv(input_file_path, header=0)

                # Step 2: Initialize an empty list to store results
                results = []

                # Step 3: Count the number of positives for each Cohort and each AJCC Stage
                for tumor_type in tumor_types:
                    # Filter data for the current Cohort
                    tumor_data = new_data[new_data['Cohort'] == tumor_type]

                    # Count positives for each AJCC stage (I, II, III)
                    for stage in ['I', 'II', 'III']:
                        stage_data = tumor_data[tumor_data['AJCC Stage'] == stage]

                        # Count positives for 'Cancer tissue' column
                        positive_count = stage_data[stage_data['Cancer tissue'] == 'Positive'].shape[0]

                        # Append result to the list
                        results.append({'Cohort': tumor_type, 'AJCC Stage': stage, 'Positive Count': positive_count})

                    # Count positives for all AJCC stages combined
                    total_positive_count = tumor_data[tumor_data['Cancer tissue'] == 'Positive'].shape[0]
                    results.append({'Cohort': tumor_type, 'AJCC Stage': 'All Stages', 'Positive Count': total_positive_count})

                # Step 4: Convert the results into a DataFrame and save to CSV
                results_df = pd.DataFrame(results)
                results_df.to_csv(output_file_path, index=False)

                print(f"Processed: {input_file_path} → Saved: {output_file_path}")

            except FileNotFoundError:
                print(f"File not found: {input_file_path}, skipping...")

        # Merge all positives_countDet CSV files into Positives1.xlsx
        # Function for natural sorting (e.g., 1,2,3 instead of 1,10,100)
        def natural_sort_key(text):
            return [int(num) if num.isdigit() else num for num in re.split(r'(\d+)', text)]

        # Get a sorted list of all relevant CSV files
        csv_files = sorted(
            [f for f in os.listdir(positives_dir) if f.startswith("positives_countDet") and f.endswith(".csv")],
            key=natural_sort_key
        )

        # Initialize an empty list to store DataFrames
        dfs = []

        # Loop through each file and process it
        for i, file in enumerate(csv_files, start=1):
            if i > num_combinations:
                break  # Stop after processing the specified number of combinations
            
            # Read the current CSV file
            file_path = os.path.join(positives_dir, file)
            df = pd.read_csv(file_path)

            # Filter rows based on conditions
            df = df[(df["AJCC Stage"] == "All Stages") & (df["Cohort"].isin(tumor_types))]

            # Rename the "Positive Count" column to include the file index
            df.rename(columns={"Positive Count": f"Positive Det {i}"}, inplace=True)

            # Append the processed DataFrame to the list
            dfs.append(df)

        # Merge all DataFrames on "Cohort" and "AJCC Stage"
        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = pd.merge(merged_df, df, on=["Cohort", "AJCC Stage"], how="outer")

        # Sort the final DataFrame by "Cohort" and "AJCC Stage" for consistency
        merged_df = merged_df.sort_values(by=["Cohort", "AJCC Stage"]).reset_index(drop=True)

        # Add "Total Positive Patients" column
        merged_df["Total Positive Patients"] = merged_df["Cohort"].map(total_positive_patients)

        # Reorder columns to place "Total Positive Patients" after "AJCC Stage"
        cols = ["Cohort", "AJCC Stage", "Total Positive Patients"] + [col for col in merged_df.columns if col.startswith("Positive Det")]
        merged_df = merged_df[cols]

        # Save the merged DataFrame to an Excel file
        output_file_path = os.path.join(positives_dir, "Positives1.xlsx")
        merged_df.to_excel(output_file_path, index=False, engine='openpyxl')

        print(f"Merged file saved as: {output_file_path}")

    def calculate_sensitivity_and_accuracy(output_dir, tumor_types, total_positive_patients, num_combinations):
        # Load data from Excel files
        positives_file_path = os.path.join(output_dir, 'Positives', 'Positives1.xlsx')
        averages_file_path = os.path.join(output_dir, 'Average1.xlsx')

        positives_df = pd.read_excel(positives_file_path)
        averages_df = pd.read_excel(averages_file_path)

        # Get the list of Positive Count columns
        positive_count_columns = [col for col in positives_df.columns if col.startswith("Positive Det")]

        # Initialize columns for results
        results = {"Sensitivity": [], "Accuracy": [], "Cohort": []}

        # Total number of positive patients across all Cohorts
        total_patients = positives_df["Total Positive Patients"].sum()

        # Loop through each Positive Count column
        for i, column in enumerate(positive_count_columns, start=1):
            if i > num_combinations:
                break  # Stop after processing the specified number of combinations
            
            # Determine the Cohort type based on the highest value in the corresponding "Average Distance %(3)-i" column
            avg_column = f"Average Distance %(3)-{i}"
            
            if avg_column not in averages_df.columns:
                print(f"Column {avg_column} not found in Averages file, skipping...")
                continue  # Skip this iteration if column is missing
            
            max_value_row = averages_df[avg_column].idxmax()
            Cohort_type = averages_df.loc[max_value_row, "Cohort"]

            # Ensure the Cohort exists in positives_df
            Cohort_row = positives_df[positives_df["Cohort"] == Cohort_type]
            if Cohort_row.empty:
                print(f"Cohort {Cohort_type} not found in Positives file, skipping...")
                continue
            
            Cohort_row = Cohort_row.iloc[0]  # Convert to Series
            
            # Extract necessary values
            TP = Cohort_row[column]  # True Positives
            FN = Cohort_row["Total Positive Patients"] - TP  # False Negatives
            FP = positives_df[column].sum() - TP  # False Positives
            TN = total_patients - (TP + FP + FN)  # True Negatives

            # Calculate Sensitivity and Accuracy
            sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
            
            # Append results
            results["Sensitivity"].append(sensitivity)
            results["Accuracy"].append(accuracy)
            results["Cohort"].append(Cohort_type)

            # Write results into the DataFrame
            positives_df.loc[positives_df["Cohort"] == Cohort_type, f"Sensitivity {i}"] = sensitivity
            positives_df.loc[positives_df["Cohort"] == Cohort_type, f"Accuracy {i}"] = accuracy

        # Save the results back to the Excel file
        positives_df.to_excel(positives_file_path, index=False, engine='openpyxl')

        print(f"Results saved to: {positives_file_path}")

    def get_top_5_sensitivities(output_dir, tumor_types, num_combinations):
        excel_file_path = os.path.join(output_dir, 'Positives', 'Positives1.xlsx')
        csv_file_folder = output_dir
        output_path = os.path.join(output_dir, 'Positives')

        # Read the Excel file
        excel_df = pd.read_excel(excel_file_path)

        # Function for natural sorting (e.g., 1,2,3 instead of 1,10,100)
        def natural_sort_key(text):
            return [int(num) if num.isdigit() else num for num in re.split(r'(\d+)', text)]

        # Function to get selected biomarkers from the appropriate CSV file
        def get_selected_biomarkers(csv_file_name, col_start=1, col_end=4):
            csv_file_path = os.path.join(csv_file_folder, csv_file_name)
            if os.path.exists(csv_file_path):
                csv_df = pd.read_csv(csv_file_path)
                return csv_df.iloc[-1, col_start:col_end].values.tolist()  # Convert NumPy array to Python list
            else:
                print(f"Warning: File {csv_file_name} not found.")
                return ["NA"]  # Return "NA" instead of an empty list

        # Initialize a list to store results
        results = []

        # Process each Cohort (row) in the Excel file
        for idx, row in excel_df.iterrows():
            Cohort = row['Cohort']
            print(f"Processing Cohort: {Cohort}")
            
            # Initialize the list for this Cohort
            Cohort_results = {
                'Cohort': Cohort,
                'Top Sensitivity 1': 'NA',
                'Top Accuracy 1': 'NA',
                'Top Biomarkers 1': 'NA',
                'Top Sensitivity 2': 'NA',
                'Top Accuracy 2': 'NA',
                'Top Biomarkers 2': 'NA',
                'Top Sensitivity 3': 'NA',
                'Top Accuracy 3': 'NA',
                'Top Biomarkers 3': 'NA',
                'Top Sensitivity 4': 'NA',
                'Top Accuracy 4': 'NA',
                'Top Biomarkers 4': 'NA',
                'Top Sensitivity 5': 'NA',
                'Top Accuracy 5': 'NA',
                'Top Biomarkers 5': 'NA'
            }
            
            # Generate ordered Sensitivity-Accuracy column pairs
            sensitivity_accuracy_pairs = sorted(
                [(f'Sensitivity {i}', f'Accuracy {i}') for i in range(1, num_combinations + 1)], 
                key=lambda x: int(x[0].split(' ')[1])  # Sort by integer value
            )
            
            # Store valid sensitivity-accuracy pairs
            valid_sensitivities = []
            
            for sensitivity_col, accuracy_col in sensitivity_accuracy_pairs:
                if sensitivity_col not in row or accuracy_col not in row:
                    continue  # Skip missing columns
                
                sensitivity_value = row[sensitivity_col]
                accuracy_value = row[accuracy_col]
                
                # Only consider Sensitivity if Accuracy > 0.60
                if pd.notna(accuracy_value) and accuracy_value > 0.60:
                    # Determine which CSV file to read based on the Sensitivity number
                    sensitivity_num = int(sensitivity_col.split(' ')[1])
                    csv_file_name = f'FeatureVectorTissueDet{sensitivity_num}.csv'
                    
                    # Get selected biomarkers for the current sensitivity
                    biomarkers = get_selected_biomarkers(csv_file_name)
                    
                    # Store the result
                    valid_sensitivities.append({
                        'Sensitivity': sensitivity_value,
                        'Accuracy': accuracy_value,
                        'Biomarkers': biomarkers
                    })
            
            # If valid sensitivities exist, get the top 5 sorted by highest sensitivity
            if valid_sensitivities:
                sorted_sensitivities = sorted(valid_sensitivities, key=lambda x: x['Sensitivity'], reverse=True)[:5]
                
                # Update the Cohort result with the top 5 sensitivities
                for i, res in enumerate(sorted_sensitivities):
                    Cohort_results[f'Top Sensitivity {i + 1}'] = res['Sensitivity']
                    Cohort_results[f'Top Accuracy {i + 1}'] = res['Accuracy']
                    Cohort_results[f'Top Biomarkers {i + 1}'] = res['Biomarkers']
            
            # Append the result for the current Cohort
            results.append(Cohort_results)

        # Create a DataFrame for the results
        results_df = pd.DataFrame(results)

        # Ensure output path exists
        os.makedirs(output_path, exist_ok=True)

        # Define output file path
        output_file_path = os.path.join(output_path, 'Top_5_Sensitivities.xlsx')

        # Save the results to Excel
        results_df.to_excel(output_file_path, index=False, engine='openpyxl')

        print(f"Results saved to {output_file_path}")

    # ==================================================
    # Main Execution
    # ==================================================

    # Part 1: Preliminary Analysis and Formation of Average1.xlsx
    process_clinical_data(file_path, sheet_name, biomarkers, output_dir, tumor_types, total_positive_patients, num_combinations, random_patient_range)

    # Part 2: Formation of Positives1.xlsx and Sensitivity Outcomes
    merge_positives_to_excel(output_dir, cohorts_to_analyze, num_combinations)
    calculate_sensitivity_and_accuracy(output_dir, cohorts_to_analyze, total_positive_patients, num_combinations)
    get_top_5_sensitivities(output_dir, cohorts_to_analyze, num_combinations)

# Example usage
file_path = r'D:\Cancer Detection folder\Test\Clinical cancer data.xlsx'
sheet_name = 'Normal and Cancer'
biomarkers = ['Angiopoietin-2', 'CA-125', 'CA 15-3', 'CEA', 'CYFRA 21-1', 'FGF2', 'G-CSF', 'HE4', 'HGF', 'PAR', 'sPECAM-1', 'Thrombospondin-2']
output_dir = r"D:\Cancer Detection folder\Test"

# User-defined inputs
tumor_types = {
    'normal': 528,
    'breast': 58,
    'colorectum': 137,
    'esophagus': 18,
    'liver': 22,
    'lung': 35,
    'ovary': 31,
    'pancreas': 32,
    'stomach': 41
}
total_positive_patients = {
    "Breast": 77, "Colorectum": 182, "Esophagus": 25, "Liver": 29,
    "Lung": 47, "Ovary": 41, "Pancreas": 42, "Stomach": 55
}
cohorts_to_analyze = ['Breast', 'Colorectum', 'Esophagus', 'Liver', 'Lung', 'Ovary', 'Pancreas', 'Stomach']
num_combinations = 220  # User-defined number of combinations
random_patient_range = 1802  # User-defined random patient range

# Execute the TDA_cancerDetect function
TDAcancer_detect_2(file_path, sheet_name, biomarkers, output_dir, tumor_types, total_positive_patients, num_combinations, random_patient_range, cohorts_to_analyze)

Processing combination 1: ['Angiopoietin-2', 'CA-125', 'CA 15-3']


C:\Users\Sudarshan Gogoi\AppData\Local\Temp\ipykernel_21384\934484306.py:243: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_combined, selected_biomarkers_row])


Saved: D:\Cancer Detection folder\Test\FeatureVectorTissueDet1.csv
Processing combination 2: ['Angiopoietin-2', 'CA-125', 'CEA']


C:\Users\Sudarshan Gogoi\AppData\Local\Temp\ipykernel_21384\934484306.py:243: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_combined, selected_biomarkers_row])


Saved: D:\Cancer Detection folder\Test\FeatureVectorTissueDet2.csv
Processing combination 3: ['Angiopoietin-2', 'CA-125', 'CYFRA 21-1']


...

Saved: D:\Cancer Detection folder\Test\FeatureVectorTissueDet219.csv
Processing combination 220: ['PAR', 'sPECAM-1', 'Thrombospondin-2']


C:\Users\Sudarshan Gogoi\AppData\Local\Temp\ipykernel_21384\934484306.py:243: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_combined, selected_biomarkers_row])


Saved: D:\Cancer Detection folder\Test\FeatureVectorTissueDet220.csv
Processed and updated D:\Cancer Detection folder\Test\FeatureVectorTissueDet1.csv and D:\Cancer Detection folder\Test\AverageDet1.csv successfully.
Processed and updated D:\Cancer Detection folder\Test\FeatureVectorTissueDet2.csv and D:\Cancer Detection folder\Test\AverageDet2.csv successfully.
Processed and updated D:\Cancer Detection folder\Test\FeatureVectorTissueDet3.csv and D:\Cancer Detection folder\Test\AverageDet3.csv successfully.
Processed and updated D:\Cancer Detection folder\Test\FeatureVectorTissueDet4.csv and D:\Cancer Detection folder\Test\AverageDet4.csv successfully.
Processed and updated D:\Cancer Detection folder\Test\FeatureVectorTissueDet5.csv and D:\Cancer Detection folder\Test\AverageDet5.csv successfully.
Processed and updated D:\Cancer Detection folder\Test\FeatureVectorTissueDet6.csv and D:\Cancer Detection folder\Test\AverageDet6.csv successfully.
Processed and updated D:\Cancer Detection f

C:\Users\Sudarshan Gogoi\AppData\Local\Temp\ipykernel_21384\934484306.py:415: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  positives_df.loc[positives_df["Cohort"] == Cohort_type, f"Sensitivity {i}"] = sensitivity
C:\Users\Sudarshan Gogoi\AppData\Local\Temp\ipykernel_21384\934484306.py:416: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  positives_df.loc[positives_df["Cohort"] == Cohort_type, f"Accuracy {i}"] = accuracy
C:\Users\Sudarshan Gogoi\AppData\Local\Temp\ipykernel_21384\934484306.py:415: PerformanceWarning: DataFrame is h

Results saved to: D:\Cancer Detection folder\Test\Positives\Positives1.xlsx
Processing Cohort: Breast
Processing Cohort: Colorectum
Processing Cohort: Esophagus
Processing Cohort: Liver
Processing Cohort: Lung
Processing Cohort: Ovary
Processing Cohort: Pancreas
Processing Cohort: Stomach
Results saved to D:\Cancer Detection folder\Test\Positives\Top_5_Sensitivities.xlsx
